In [4]:
include("StandardCode.jl")

bs_bf (generic function with 1 method)

In [7]:

function TSSs.TSS_raw_quant(t2, tss_folder, image_folder, n; xy = 0.189, zx = 0.5)
    
    images_pat = intersect(TSSs.get_image_patterns(t2), TSSs.get_image_patterns(tss_folder))
    
    p = Progress(length(images_pat), 1)

    dfs = []

    for a in 1:length(images_pat)
        next!(p)
        

        pat = images_pat[a]

        # Get all the TSS in the images
        a = TSSs.find_outline(tss_folder, pat)
        tss = TSSs.cell_tss_dict(a)
        tss = TSSs.meassure_tss(tss, imagesfolder, pat, n, xy = xy, zx = zx)
    
        df = TSSs.analysis_singlechannel(tss; image = pat,  tss_name = :TSS2)
        push!(dfs, df)

    end
    
    d = TSSs.join_in_all_common_columns(dfs...)
    return d
    
end

function normalise_avgdot(df, genename, expn, typeprobe, ibidislide; root = pwd(), genefolder = "Gene")
    dot1_r1 = TSSs.int_brightest_pixel(TSSs.read_tiff_as_gray(root * typeprobe *"/_mRNA_AVG_ns.tif").*255; radious = 1)
    dot1_r2 = TSSs.int_brightest_pixel(TSSs.read_tiff_as_gray(root * typeprobe * "/_mRNA_AVG_ns.tif").*255; radious = 2)
    df[!,:TSS1_r1] = df[!,:locus1_int1_TSS2] ./ dot1_r1
    df[!,:TSS1_r2] = df[!,:locus1_int2_TSS2] ./ dot1_r2
    df[!,:TSS2_r1] = df[!,:locus2_int1_TSS2] ./ dot1_r1
    df[!,:TSS2_r2] = df[!,:locus2_int2_TSS2] ./ dot1_r2
    CSV.write("TSS_avgdot/"*genename*"_exp"*"$expn"*".csv", df)
    ibidislide[!,:Well] = [split(ii, " (")[1] for ii in ibidislide[!,:Well]]
    cp_dir = root1*"CP_results"
    cells1 = CellInfo(cp_dir, add_probetypes(exp));
    exp1 = CSV.read("TSS_avgdot/"*gene*"_exp"*"$expn"*".csv", DataFrame)
    exp1[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in exp1[!,:Image]]
    exp1 = innerjoin(exp1, ibidislide, on= "Well")
    CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_TSS.csv", exp1)
    CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_CP.csv", cells1)
    fq_dir1 = root1*typeprobe*"/"
    fq1 = FQ_summary_MATURE(fq_dir1)
    CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_FQ.csv", fq1)
end


normalise_avgdot (generic function with 1 method)

In [1]:
ROOT1 = "/Volumes/lymphdev\$/IreneR/Confocal/peliinex/Rep1_Day2/"
ROOT2 = "/Volumes/lymphdev\$/IreneR/Confocal/peliinex/Rep2/"
ROOT3 = "/Volumes/lymphdev\$/IreneR/Confocal/peliinex/Rep3/"

"/Volumes/lymphdev\$/IreneR/Confocal/peliinex/Rep3/"

In [25]:
root1 = ROOT1
expn = 1
gene = "Peli1_intron_original"
typeprobe = "type4"
tss_c4 = root1 * "Segmentation_type4/_FQ_outline/_TS_detect"
imagesfolder = root1 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)
d = TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 3; xy = 0.189, zx = 0.5)
CSV.write("TSS_raw/"*gene*"_exp"*"$expn"*".csv", d)

Progress: 100%|█████████████████████████████████████████| Time: 0:50:22


"TSS_raw/Peli1_intron_original_exp1.csv"

In [26]:
root1 = ROOT2
expn = 2
gene = "Peli1_intron_original"
typeprobe = "type4"
tss_c4 = root1 * "Segmentation_type4/_FQ_outline/_TS_detect"
imagesfolder = root1 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)
d = TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 3; xy = 0.189, zx = 0.5)
CSV.write("TSS_raw/"*gene*"_exp"*"$expn"*".csv", d)

Progress: 100%|█████████████████████████████████████████| Time: 1:18:47


"TSS_raw/Peli1_intron_original_exp2.csv"

In [27]:
root1 = ROOT3
expn = 3
gene = "Peli1_intron_original"
tss_c4 = root1 * "Segmentation_type4/_FQ_outline/_TS_detect"
imagesfolder = root1 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)
d = TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 3; xy = 0.189, zx = 0.5)
CSV.write("TSS_raw/"*gene*"_exp"*"$expn"*".csv", d)

Progress: 100%|█████████████████████████████████████████| Time: 0:52:13


"TSS_raw/Peli1_intron_original_exp3.csv"

In [8]:
folder = "Peli1_original"

root1 = ROOT1
exp = CSV.read("IbidiChambers/IbidiChamberSlide_Peli1original_1.csv", DataFrame)
expn = 1
gene = "Peli1_intron_original"
typeprobe = "type4"
tss1 = CSV.read("TSS_raw/"*gene*"_exp"*"$expn"*".csv", DataFrame)
normalise_avgdot(tss1, gene, expn, typeprobe, exp; root = root1, genefolder = folder)

root1 = ROOT2
exp = CSV.read("IbidiChambers/IbidiChamberSlide_Peli1original_2.csv", DataFrame)
expn = 2
gene = "Peli1_intron_original"
typeprobe = "type4"
tss1 = CSV.read("TSS_raw/"*gene*"_exp"*"$expn"*".csv", DataFrame)
normalise_avgdot(tss1, gene, expn, typeprobe, exp; root = root1, genefolder = folder)

root1 = ROOT3
exp = CSV.read("IbidiChambers/IbidiChamberSlide_Peli1original_3.csv", DataFrame)
expn = 3
gene = "Peli1_intron_original"
typeprobe = "type4"
tss1 = CSV.read("TSS_raw/"*gene*"_exp"*"$expn"*".csv", DataFrame)
normalise_avgdot(tss1, gene, expn, typeprobe, exp; root = root1, genefolder = folder)


"Peli1_original/Peli1_intron_original_exp3_FQ.csv"

In [9]:
genefolder = "Peli1_original"
root1 = ROOT1
expn = 1
typeprobe = "type6"
exp = CSV.read("IbidiChambers/IbidiChamberSlide_Peli1original_1.csv", DataFrame)

fq_dir1 = root1*typeprobe*"/"
fq1 = FQ_summary_MATURE(fq_dir1)
CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_FQ_mature.csv", fq1)

root1 = ROOT2
exp = CSV.read("IbidiChambers/IbidiChamberSlide_Peli1original_2.csv", DataFrame)
expn = 2

typeprobe = "type6"
fq_dir1 = root1*typeprobe*"/"
fq1 = FQ_summary_MATURE(fq_dir1)
CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_FQ_mature.csv", fq1)

root1 = ROOT3
exp = CSV.read("IbidiChambers/IbidiChamberSlide_Peli1original_3.csv", DataFrame)
expn = 3
typeprobe = "type6"
fq_dir1 = root1*typeprobe*"/"
fq1 = FQ_summary_MATURE(fq_dir1)
CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_FQ_mature.csv", fq1)


"Peli1_original/Peli1_intron_original_exp3_FQ_mature.csv"